In [1]:
import os
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px

from twarc import Twarc2, expansions

from dotenv import load_dotenv

load_dotenv()
from data import load_data, get_time_interval

In [2]:
USER = "nicktorba"
HOURS = 24

end_time, start_time = get_time_interval(HOURS)
print(end_time, "    ", start_time)

2022-06-22T13:27:09Z      2022-06-21T13:27:09Z


In [35]:
from twarc_csv import DataFrameConverter
converter = DataFrameConverter("tweets", inline_referenced_tweets=False)

help(DataFrameConverter)

Help on class DataFrameConverter in module dataframe_converter:

class DataFrameConverter(builtins.object)
 |  DataFrameConverter(input_data_type='tweets', json_encode_all=False, json_encode_text=False, json_encode_lists=True, inline_referenced_tweets=False, merge_retweets=True, allow_duplicates=False, extra_input_columns='', output_columns=None, dataset_ids=None, counts=None)
 |  
 |  Convert a set of JSON Objects into a Pandas DataFrame object.
 |  You can call this directly on a small set of objects, but memory is quickly consumed for larger datasets.
 |  
 |  This class can accept individual tweets or whole response objects.
 |  
 |  Args:
 |      objects (iterable): JSON Objects to convert. Can be users, tweets, or other API objects.
 |      input_data_type (str): data type: `tweets` or `users` or `compliance` or `counts`
 |  Returns:
 |      DataFrame: The objects provided as a Pandas DataFrame.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, input_data_type='tweets', json

In [7]:
client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)

df_following, df_f_all_tweets, df_f_all_mentions = load_data(client, USER, start_time, end_time, pull_new=False)

df_f_all_mentions["tweet_link"] = df_f_all_mentions.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)
df_f_all_tweets["tweet_link"] = df_f_all_tweets.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)


/home/nick/Documents/GitHub/tweetscape-experiments/notebooks/reply-views/data.py:30: DtypeWarning: Columns (60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_f_all_tweets = pd.read_csv(f"data/{username}/{latest_tweet_data}")


In [8]:
tweeter = "goblinodds" 

df_tweeter = df_f_all_tweets[df_f_all_tweets["author.username"] == tweeter]

df_tweeter["public_metrics.reply_count"].value_counts()

0.0      88
1.0      75
2.0      17
3.0      10
4.0       6
5.0       3
25.0      2
47.0      1
18.0      1
6.0       1
32.0      1
9.0       1
20.0      1
103.0     1
38.0      1
51.0      1
11.0      1
44.0      1
15.0      1
76.0      1
86.0      1
Name: public_metrics.reply_count, dtype: int64

In [28]:
df_tweeter[df_tweeter["public_metrics.reply_count"] == 86].iloc[0]["tweet_link"]
# df_tweeter[df_tweeter["public_metrics.reply_count"] == 86].iloc[0]["id"]

'https://twitter.com/goblinodds/status/1488356526028771329'

In [25]:
df_f_all_mentions["id"].dtype

dtype('int64')

In [27]:
df_f_all_mentions[df_f_all_mentions["referenced_tweets.replied_to.id"] == 1488356526028771329]

,Unnamed: 0,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,...,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 74,tweet_link


In [32]:
all_mentioned_tweet_ids = df_f_all_mentions["referenced_tweets.replied_to.id"].unique()

for i in all_mentioned_tweet_ids:
    if i in df_f_all_tweets["id"]:
        print("true")

        

In [33]:
df_f_all_mentions

,Unnamed: 0,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,...,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 74,tweet_link
0,0,1538875504866086912,1538721995126296578,1.538726e+18,NaN,NaN,323436887,709644992.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:08:32+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/jakesing_/status/153887550...
1,1,1538870530786619392,1534336904845479936,1.534340e+18,NaN,NaN,3147878535,709644992.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:08:32+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/EternleNotes/status/153887...
2,2,1538842804130136065,1538721995126296578,1.538726e+18,NaN,NaN,1527694420535300097,709644992.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:08:32+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/alvinbnet/status/153884280...
3,3,1538839586947858432,1538745961517752320,1.538748e+18,NaN,NaN,3415881575,709644992.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:08:32+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/minney_cat/status/15388395...
4,4,1538838288076886017,1534336904845479936,1.534337e+18,NaN,NaN,1526570350707884032,709644992.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:08:32+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/MarcValle17/status/1538838...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11939,10,1538619805489561600,1538617637466386433,1.538618e+18,NaN,NaN,835545646094401536,68746721.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:51:04+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/PhilsburyDoboy/status/1538...
11940,11,1538618824563499010,1538617637466386433,1.538618e+18,NaN,NaN,937108761973608453,68746721.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:51:04+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/fbranchaud1/status/1538618...
11941,12,1538606403463569409,1538606403463569409,NaN,1.536860e+18,NaN,423040124,NaN,33836629.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:51:04+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/kraus_read/status/15386064...
11942,13,1538606358634938369,1538606358634938369,NaN,1.536839e+18,NaN,423040124,NaN,44196397.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2022-06-20T15:51:04+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/kraus_read/status/15386063...
